In [7]:
!pip uninstall torch torchvision torchaudio xformers -y

In [2]:
import torch, torchvision, xformers
print(torch.__version__, torch.version.cuda)
print(torchvision.__version__)
print('xformers built for:', xformers.version.cuda_version)

ModuleNotFoundError: No module named 'xformers'

In [3]:
!pip install -q transformers torchinfo datasets fsspec huggingface_hub evaluate xformers

In [8]:
!pip install torch==2.7.0 torchvision==0.22.0 torchaudio==2.7.0 --index-url https://download.pytorch.org/whl/cu126

# Install other packages
!pip install -q transformers torchinfo datasets fsspec huggingface_hub evaluate xformers

Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.0/867.0 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 60.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0+cu126 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 73.3 MB/s eta 0:00:00
^C


In [6]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.7.0+cu126
CUDA available: True
CUDA version: 12.6


In [5]:
!nvidia-smi

Thu Jun 12 00:21:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             42W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
import numpy as np
import transformers
import evaluate
import torch, torchvision, torchaudio
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from torchinfo import summary
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertModel, BertForSequenceClassification,AutoModelForCausalLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
import re
import xformers

In [5]:
nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert_model = AutoModel.from_pretrained('bert-base-cased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [7]:
neo_bert_tokenizer = AutoTokenizer.from_pretrained("chandar-lab/NeoBERT")
neo_bert_model = AutoModel.from_pretrained("chandar-lab/NeoBERT")

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/928 [00:00<?, ?B/s]

...

model.py:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

rotary.py:   0%|          | 0.00/2.58k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/chandar-lab/NeoBERT:
- rotary.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/chandar-lab/NeoBERT:
- model.py
- rotary.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


The repository `chandar-lab/NeoBERT` contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/chandar-lab/NeoBERT.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


model.safetensors:   0%|          | 0.00/981M [00:00<?, ?B/s]

In [14]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `model` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `model`


In [15]:
llama_model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [16]:
dataset = load_dataset("allenai/c4", "en", split="train", streaming = True)
samples = []
for i, example in enumerate(dataset):
    samples.append(example)
    if i == 10:
        break
print(samples[0])

README.md:   0%|          | 0.00/41.1k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

{'text': 'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.\nHe will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.\nThe cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.', 'timestamp': '2019-04-25 12:57:54', 'url': 'https://klyq.com/beginners-bbq-class-taking-place-in-missoula/'}


In [17]:
texts = [entry['text'] for entry in samples]

In [18]:
def clean_text(text):
    # Remove backslashes and quotes
    text = re.sub(r"[\\\'\"]", "", text)

    # Replace multiple dots/commas/periods with a single one
    text = re.sub(r"\.{2,}", ".", text)
    text = re.sub(r",+", ",", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)

    # Remove any non-printable or control characters
    text = re.sub(r"[^\x20-\x7E]", "", text)

    return text.strip()


In [19]:
cleaned_texts = [clean_text(t) for t in texts]

In [20]:
# Split each cleaned text into sentences
sentence_splits = [nltk.sent_tokenize(text) for text in cleaned_texts]

sentence_splits= [sent for sublist in sentence_splits for sent in sublist]

In [21]:
texts = sentence_splits

In [23]:
testtext = "the cat ran up the wall"

In [28]:
neo_bert_input = neo_bert_tokenizer(testtext, return_tensors="pt")
neo_bert_embeds = neo_bert_model(**neo_bert_input).last_hidden_state
llama_emb_layer = llama_model.get_input_embeddings()
llama_emb_dim = llama_emb_layer.embedding_dim

In [30]:
llama_dim = llama_model.model.embed_tokens.embedding_dim
llama_dim

4096

In [27]:
neo_bert_embeds.shape

torch.Size([1, 8, 768])

In [31]:
bert_dim = 768
hidden_dim = 2048
llama_dim = llama_model.model.embed_tokens.embedding_dim
dropout_rate = 0.1

In [32]:
projector = nn.Sequential(
    nn.LayerNorm(768),
    nn.Linear(768, 2048),
    nn.GELU(),
    nn.Dropout(0.1),
    nn.LayerNorm(2048),
    nn.Linear(2048, llama_dim),
    nn.LayerNorm(llama_dim)
)

#projected_output = projector(bert_embeds)

In [33]:
optimizer = torch.optim.AdamW(projector.parameters(), lr=1e-4)

In [34]:
num_epochs = 10

for epoch in range(num_epochs):
    total_loss = 0.0

    for text in texts:
        optimizer.zero_grad()

        # BERT input
        bert_inputs = bert_tokenizer(text, return_tensors="pt")
        bert_embeds = bert_model(**bert_inputs).last_hidden_state

        # Project to LLaMA space
        projected_output = projector(bert_embeds)

        # LLaMA labels
        target_ids = llama_tokenizer(text, return_tensors="pt").input_ids

        # Align sequence lengths
        min_len = min(projected_output.shape[1], target_ids.shape[1])
        projected_output = projected_output[:, :min_len, :]
        aligned_targets = target_ids[:, :min_len]

        # Forward + loss
        llama_output = llama_model(inputs_embeds=projected_output, labels=aligned_targets)
        loss = llama_output.loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {loss} - Avg Loss: {total_loss/len(texts):.4f}")

Epoch 1 - Loss: 6.773967742919922 - Avg Loss: 8.4737
Epoch 2 - Loss: 3.3171894550323486 - Avg Loss: 5.4678
Epoch 3 - Loss: 1.1993122100830078 - Avg Loss: 2.7597
Epoch 4 - Loss: 0.8256551623344421 - Avg Loss: 1.6166
Epoch 5 - Loss: 0.24993370473384857 - Avg Loss: 0.9423
Epoch 6 - Loss: 0.04539520666003227 - Avg Loss: 0.6483
Epoch 7 - Loss: 0.047900836914777756 - Avg Loss: 0.4391
Epoch 8 - Loss: 0.07171398401260376 - Avg Loss: 0.3315
Epoch 9 - Loss: 0.14448852837085724 - Avg Loss: 0.2977
Epoch 10 - Loss: 0.00478327414020896 - Avg Loss: 0.2351


In [37]:
sample1 = "Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared."
bert_inputs1 = bert_tokenizer(sample1, return_tensors="pt", padding=True, truncation=True)
bert_inputs1 = {k: v.to(llama_model.device) for k, v in bert_inputs1.items()}

# BERT → embeddings
with torch.no_grad():
    bert_embeds1 = bert_model(**bert_inputs1).last_hidden_state  # [1, T, 768]

# Project
with torch.no_grad():
    projected_embeds = projector(bert_embeds1)

pad_token_id = llama_tokenizer.pad_token_id or llama_tokenizer.eos_token_id


# Generate from LLaMA
generated_ids = llama_model.generate(
    inputs_embeds=projected_embeds,
    attention_mask=bert_inputs1["attention_mask"],
    max_new_tokens=30,
    do_sample=False,
    pad_token_id=pad_token_id
)


decoded_text = llama_tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated:", decoded_text)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generated: . component his, Bza, Q, E                    
